In [1]:
import sys
sys.path.append('../src')
sys.path.append('../scripts')

import warnings
warnings.filterwarnings('ignore')

In [3]:
from jobDescriptionScraper import LinkedInDescriptionScraper
from jobListingScraper import LinkedInJobListingScraper

In [4]:
from resumeRestructre import save_restructured_resume

In [11]:
# Restructuring the input resume
restructured_resume_path, restructured_resume_json= save_restructured_resume('inputs\sample_resume_input_1.pdf', 1)

In [5]:
%%time
# Job Listing Scraping
job_listing_scraper = LinkedInJobListingScraper()
jobs = job_listing_scraper.scrape_linkedin_jobs("Machine Learning Engineer", "India", 5, "../linkedincookies.pkl")

Scraping page starting at 0
CPU times: total: 172 ms
Wall time: 24.5 s


In [6]:
%%time
# Job Description Scraping
job_description_scraper = LinkedInDescriptionScraper()
jobs['Job Descriptions'] = jobs['Job ID'].apply(lambda x : job_description_scraper.getDescription(job_id=x))

CPU times: total: 31.2 ms
Wall time: 16.6 s


In [7]:
from pdfLoader import loadPdfContent
restructured_resume_path = "..\inputs\sample_resume_input_1.pdf"
old_resume_content = loadPdfContent(restructured_resume_path)

In [8]:
# Old Resume Scoring
from resumeRelevancyScore import RelevanceScorer
resume_relevence_scorer = RelevanceScorer()

jobs['Old Resume Score'] = jobs['Job Descriptions'].apply(lambda x : resume_relevence_scorer.calculate_relevance_score(old_resume_content, x))

In [9]:
jobs

,Job Title,Job Post Url,Company Name,Location,Job ID,Job Descriptions,Old Resume Score
0,Generative AI Engineer,https://www.linkedin.com/jobs/view/4009701610/...,Deloitte,"Bengaluru, Karnataka, India (Hybrid)",4009701610,"Hi, We are looking for Candidates who are havi...",66.19
1,Data and Applied Scientist II,https://www.linkedin.com/jobs/view/3977751872/...,Microsoft,"Kerala, India (Hybrid)",3977751872,Microsoft’s Cloud business is experiencing exp...,66.57
2,Machine Learning Engineer,https://www.linkedin.com/jobs/view/3992115810/...,KeyValue Software Systems,"Kochi, Kerala, India (On-site)",3992115810,About KeyValue\n\nKeyValue is a product develo...,57.31
3,Sr. Machine Learning Engineer,https://www.linkedin.com/jobs/view/4015810227/...,Acentra Health,"Chennai, Tamil Nadu, India (On-site)",4015810227,Job Title: Sr. Machine Learning Engineer\nLoca...,59.58
4,"Senior Machine Learning Engineer (NLP, ML Models)",https://www.linkedin.com/jobs/view/4009752525/...,Qlik,"Bengaluru, Karnataka, India (On-site)",4009752525,Description\n\nWhat makes us Qlik?\n\nA Gartne...,67.20


In [13]:
from reGenerate import regen_resume
import pandas as pd

def apply_regen_resume(row):
    regen_json, output_filepath = regen_resume(row['Job Descriptions'], restructured_resume_json, row['Job ID'])
    return pd.Series({'ReGen JSON': regen_json, 'ReGen FilePath': output_filepath})

jobs[['ReGen JSON', 'ReGen FilePath']] = jobs.apply(apply_regen_resume, axis=1)

In [14]:
from resumeRelevancyScore import RelevanceScorer
resume_relevence_scorer = RelevanceScorer()

# Calculating the relavancy score of new resume
jobs['New Resume Score'] = jobs[['Job Descriptions', 'Job ID']].apply(lambda x : resume_relevence_scorer.calculate_relevance_score_new(x[1], x[0]), axis=1)

In [15]:
jobs

,Job Title,Job Post Url,Company Name,Location,Job ID,Job Descriptions,Old Resume Score,ReGen JSON,ReGen FilePath,New Resume Score
0,Generative AI Engineer,https://www.linkedin.com/jobs/view/4009701610/...,Deloitte,"Bengaluru, Karnataka, India (Hybrid)",4009701610,"Hi, We are looking for Candidates who are havi...",66.19,"{'name': 'Muhammed Büyükkınacı', 'contact': {'...",../generatedResumes\4009701610.pdf,78.99
1,Data and Applied Scientist II,https://www.linkedin.com/jobs/view/3977751872/...,Microsoft,"Kerala, India (Hybrid)",3977751872,Microsoft’s Cloud business is experiencing exp...,66.57,"{'name': 'Muhammed Büyükkınacı', 'contact': {'...",../generatedResumes\3977751872.pdf,71.02
2,Machine Learning Engineer,https://www.linkedin.com/jobs/view/3992115810/...,KeyValue Software Systems,"Kochi, Kerala, India (On-site)",3992115810,About KeyValue\n\nKeyValue is a product develo...,57.31,"{'name': 'Muhammed Büyükkınacı', 'contact': {'...",../generatedResumes\3992115810.pdf,63.30
3,Sr. Machine Learning Engineer,https://www.linkedin.com/jobs/view/4015810227/...,Acentra Health,"Chennai, Tamil Nadu, India (On-site)",4015810227,Job Title: Sr. Machine Learning Engineer\nLoca...,59.58,"{'name': 'Muhammed Büyükkınacı', 'contact': {'...",../generatedResumes\4015810227.pdf,70.46
4,"Senior Machine Learning Engineer (NLP, ML Models)",https://www.linkedin.com/jobs/view/4009752525/...,Qlik,"Bengaluru, Karnataka, India (On-site)",4009752525,Description\n\nWhat makes us Qlik?\n\nA Gartne...,67.20,"{'name': 'Muhammed Büyükkınacı', 'contact': {'...",../generatedResumes\4009752525.pdf,74.46
